In [9]:
import os
import pandas as pd
import git
import numpy as np



# folder = "C:\\Users\\Van\\Desktop\\vancanwin\\pgap3cdg\\01.20.23\\"

def csv_file_adjust(folder,filename):
    filetype = filename.split(".")[-1]
    if "csv" in filetype:
        df = pd.read_csv(folder+filename, on_bad_lines='skip', encoding = "utf-8",sep="[_,\t ;|:]")#,sep ="[_\t ;|:]",engine='python')
        df.to_csv(folder+"\\"+filename.split(".")[0]+".csv")

        # df = df.iloc[:,0].str.split(",",expand=True)
        # filename = filename.split(".")[0]+".xlsx"
        # df.to_excel(folder+filename)
        # #     if "csv" in filetype:
        # # df = pd.read_csv(folder+filename, sep='delimiter',on_bad_lines='skip')
        # # df = df.iloc[:,0].str.split(",",expand=True)

def isnumber(x):
    try:
        int(x)
        return True
    except:
        return False
    
def normalize_plate384(folder,filename):
    '''Enter file to read plate and normalize data'''
    print(folder+filename)
    filetype = filename.split(".")[-1]
    df = pd.DataFrame([])

    
    df = pd.read_csv(folder+filename, on_bad_lines='skip', encoding = "utf-8",sep="[_,\t ;|:]")#,sep ="[_\t ;|:]",engine='python')
    col_num = 30#df.shape[1]
    index_start = 0
    index_end = 16
    column_index_end = 24
    df_results_index = 0
    for column in range(col_num):
        print("column number", column, type(column), col_num)
        print(df.iloc[:,column])
        try:
            df_results_index = df[df.iloc[:,column].str.contains('A', na=False)].index#].isin(['A'])].index#
            # column_header = column
            column_index = column+1
            print("line 25 matching?",df_results_index)
            break
        except Exception as e:
            print(e)
            
            pass
        # print('checking index', df.iloc[:,column])
    for i, index in enumerate(df_results_index):
        # dateset_name = df.iloc[index,0]
        new_header = df.iloc[index-1,column_index:column_index+column_index_end]
        df_results = df.iloc[index+index_start:index+index_end,column_index:column_index+column_index_end] #row,col
        df_results = df_results[df_results.applymap(isnumber)]
        count_na = df_results.isna().sum().sum()
        count_values = df_results.count().sum()

        print("NA count:", count_na, "Values count:", count_values, "done")
        try:
            if  count_na < 100 and count_values > 100:
                print("checking 36",df_results)
                df_results = df_results.fillna(0)
                df_results = df_results.astype(int)
                df_results.columns = new_header
                df_median_neg_controls = int(df_results.iloc[:,0:2].stack().median())
                df_min = df_results.stack().min()
                df_max = df_results.stack().max()
                df_range = df_max-df_min
                df_results_norm = (df_results - df_min)/df_range
                df.iloc[index+index_start:index+index_end,column_index:column_index+column_index_end] = df_results_norm
                print("processed...",i)
            else:
                print("skip data")
                pass
        except Exception as e: 
            print(e)
            pass

    # df = df.iloc[:,column_index:column_index+column_index_end]
    df.to_csv(folder+"\\completed\\"+filename.split(".")[0]+"_normalized."+filename.split(".")[-1])
    return(df)

def zscore_plate384(folder,filename):
    '''Enter file to read plate and normalize data'''
    print(folder+filename)
    filetype = "csv"
    filename = filename.split(".")[0]
    print("Filename z-score",folder+filename, filetype)

    df = pd.DataFrame([])
    df = pd.read_csv(folder+"\\completed\\"+filename+"_normalized."+filetype, encoding = "utf-8",sep=',')
    col_num = 30#df.shape[1]
    index_start = 0
    index_end = 16
    column_index_end = 24
  
    for column in range(col_num):
        print("z-score column number", column, type(column))
        try:
            df_results_index = df[df.iloc[:,column].str.contains('A', na=False,regex=False)].index
            column_index = column+1
            print("line 95 matching?",df_results_index)
            break
        except Exception as e:
            print(e)
            pass
    for i, index in enumerate(df_results_index):
        new_header = df.iloc[index-1,column_index:column_index+column_index_end]
        df_results = df.iloc[index+index_start:index+index_end,column_index:column_index+column_index_end] #row,col
        count_na = df_results.isna().sum().sum()
        count_values = df_results.count().sum()

        print("zscore NA count:", count_na, "Values count:", count_values, "done")
        try:
            if  count_na < 100 and count_values > 100:

                df_results = df_results.fillna(0)
                df_results = df_results.astype(float)

                df_results.columns = new_header
                df_min = df_results.stack().min()
                df_max = df_results.stack().max()
                df_range = df_max-df_min
                df_results_norm = (df_results - df_min)/df_range

                df_mean = df_results_norm.stack().mean()
                df_std = df_results_norm.stack().std()
                df_results_zscore = (df_results_norm - df_mean)/df_std
                df.iloc[index+index_start:index+index_end,column_index:column_index+column_index_end] = df_results_zscore
                print("zscore processed...",i)
            else:
                print("zscore skip data")
                pass
        except Exception as e: 
            print(e)
            pass
    df.to_csv(folder+"\\completed\\"+filename.split(".")[0]+"_zscore.csv")

    return(df)

def index_csv(folder,filename):
    print(folder+filename)
    filetype = "csv"
    filename = filename.split(".")[0]
    print("Filename z-score",folder+filename, filetype)

    df = pd.DataFrame([])
    df = pd.read_csv(folder+"\\completed\\"+filename+"_normalized."+filetype, encoding = "utf-8",sep=',')
    col_num = 30#df.shape[1]
    index_start = 0
    index_end = 16
    column_index_end = 24
    value_list = []
    for column in range(col_num):
        print("z-score column number", column, type(column))
        try:
            df_results_index = df[df.iloc[:,column].str.contains('A', na=False,regex=False)].index
            column_index = column+1
            print("line 95 matching?",df_results_index)
            break
        except Exception as e:
            print(e)
            pass
    for i, index in enumerate(df_results_index):
        new_header = df.iloc[index-1,column_index:column_index+column_index_end]
        df_results = df.iloc[index+index_start:index+index_end,column_index:column_index+column_index_end] #row,col
        count_na = df_results.isna().sum().sum()
        count_values = df_results.count().sum()

        print("zscore NA count:", count_na, "Values count:", count_values, "done")
        try:
            if  count_na < 100 and count_values > 100:
                print("Dimensions: ",df_results.shape[0], df_results.shape[1])
                for row in range(df_results.shape[0]):
                    row_l = chr(ord('@')+row+1)
                    for col in range(df_results.shape[1]):
                        col_l = col+1
                        print("All results: ", str(row_l)+str(col_l), df_results.iloc[row,col])
                        value_set = [str(row_l)+str(col_l), df_results.iloc[row,col]]
                        value_list.append(value_set)
        except Exception as e: 
            print(e)
            pass
    df_list = pd.DataFrame([value_list]).transpose()
    df_list.to_csv(folder+"\\completed\\"+filename.split(".")[0]+"_list.csv")

        
repo = git.Repo('.', search_parent_directories=True)
folder = repo.working_tree_dir +"\\yeast-data\\"

plate_list = os.listdir(folder)
print(plate_list,folder)

for file in plate_list:
    filetype = file.split(".")[-1]
    if filetype == "csv":
        print("Filename:", file)
        csv_file_adjust(folder,file)
        normalize_plate384(folder,file)
        zscore_plate384(folder,file)
        index_csv(folder,file)

       
    else:
        print(filetype)
        pass

   

['1.csv', '10.csv', '11.csv', '12.csv', '13.csv', '14.csv', '15.csv', '16.csv', '17.csv', '18.csv', '19.csv', '2.csv', '20.csv', '21.csv', '22.csv', '23.csv', '24.csv', '25.csv', '26.csv', '27.csv', '3.csv', '4.csv', '5.csv', '6.csv', '7.csv', '8.csv', '9.csv', 'archive', 'completed'] C:\Users\vandu\Desktop\vancanwin\yeast-data\
Filename: 1.csv
C:\Users\vandu\Desktop\vancanwin\yeast-data\1.csv
column number 0 <class 'int'> 30
0        0
1        1
2        2
3        3
4        4
      ... 
117    117
118    118
119    119
120    120
121    121
Name: Unnamed: 0, Length: 122, dtype: int64
Can only use .str accessor with string values!
column number 1 <class 'int'> 30
0        0
1        1
2        2
3        3
4        4
      ... 
117    117
118    118
119    119
120    120
121    121
Name: Unnamed, Length: 122, dtype: int64
Can only use .str accessor with string values!
column number 2 <class 'int'> 30
0        0
1        1
2        2
3        3
4        4
      ... 
117    117
118   

C:\Users\vandu\AppData\Local\Temp\ipykernel_16724\1409865649.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(folder+filename, on_bad_lines='skip', encoding = "utf-8",sep="[_,\t ;|:]")#,sep ="[_\t ;|:]",engine='python')
C:\Users\vandu\AppData\Local\Temp\ipykernel_16724\1409865649.py:37: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(folder+filename, on_bad_lines='skip', encoding = "utf-8",sep="[_,\t ;|:]")#,sep ="[_\t ;|:]",engine='python')


zscore NA count: 51 Values count: 21 done
zscore NA count: 28 Values count: 20 done
Filename: 10.csv
C:\Users\vandu\Desktop\vancanwin\yeast-data\10.csv
column number 0 <class 'int'> 30
0        0
1        1
2        2
3        3
4        4
      ... 
117    117
118    118
119    119
120    120
121    121
Name: Unnamed: 0, Length: 122, dtype: int64
Can only use .str accessor with string values!
column number 1 <class 'int'> 30
0        0
1        1
2        2
3        3
4        4
      ... 
117    117
118    118
119    119
120    120
121    121
Name: Unnamed, Length: 122, dtype: int64
Can only use .str accessor with string values!
column number 2 <class 'int'> 30
0        0
1        1
2        2
3        3
4        4
      ... 
117    117
118    118
119    119
120    120
121    121
Name: Unnamed: 2, Length: 122, dtype: int64
Can only use .str accessor with string values!
column number 3 <class 'int'> 30
0              Plate
1                  1
2         Background
3              Plate

C:\Users\vandu\AppData\Local\Temp\ipykernel_16724\1409865649.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(folder+filename, on_bad_lines='skip', encoding = "utf-8",sep="[_,\t ;|:]")#,sep ="[_\t ;|:]",engine='python')
C:\Users\vandu\AppData\Local\Temp\ipykernel_16724\1409865649.py:37: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(folder+filename, on_bad_lines='skip', encoding = "utf-8",sep="[_,\t ;|:]")#,sep ="[_\t ;|:]",engine='python')


zscore NA count: 329 Values count: 55 done
zscore skip data
zscore NA count: 333 Values count: 51 done
zscore skip data
zscore NA count: 334 Values count: 50 done
zscore skip data
zscore NA count: 108 Values count: 276 done
zscore skip data
zscore NA count: 323 Values count: 61 done
zscore skip data
zscore NA count: 332 Values count: 52 done
zscore skip data
zscore NA count: 338 Values count: 46 done
zscore skip data
zscore NA count: 309 Values count: 51 done
zscore skip data
zscore NA count: 51 Values count: 21 done
zscore skip data
zscore NA count: 28 Values count: 20 done
zscore skip data
C:\Users\vandu\Desktop\vancanwin\yeast-data\10.csv
Filename z-score C:\Users\vandu\Desktop\vancanwin\yeast-data\10 csv
z-score column number 0 <class 'int'>
Can only use .str accessor with string values!
z-score column number 1 <class 'int'>
Can only use .str accessor with string values!
z-score column number 2 <class 'int'>
Can only use .str accessor with string values!
z-score column number 3 <cl

...

line 25 matching? Int64Index([7, 24, 25, 26, 27, 60, 79, 89, 103, 107, 119, 120], dtype='int64')
NA count: 0 Values count: 384 done
checking 36    Unnamed.69 Unnamed.70 Unnamed: 6      2    0.2 Unnamed.34 Unnamed.35  \
7       12760      13520      15160  12880  11880      14600      14440   
8       13960      12240      13840  12480  13400      13400      15040   
9       14840      14560      14720  15440  13600      15040      15200   
10      13680      13360      14480  10840  15000      15160      15360   
11      14040      15960      11680  15120  12480      15480      15840   
12      13520      15280      15640  12680  14160      16240      14960   
13      13880      14560      25760  14360  14880      16520      15600   
14      14560      13920      11320  13880  12320      15520      14720   
15      14160      14320      15000  15240  14520      16560      15680   
16      13960      13360      13880  15640  14800      12680      15800   
17      14200      14360      1

C:\Users\vandu\AppData\Local\Temp\ipykernel_16724\1409865649.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(folder+filename, on_bad_lines='skip', encoding = "utf-8",sep="[_,\t ;|:]")#,sep ="[_\t ;|:]",engine='python')
C:\Users\vandu\AppData\Local\Temp\ipykernel_16724\1409865649.py:37: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(folder+filename, on_bad_lines='skip', encoding = "utf-8",sep="[_,\t ;|:]")#,sep ="[_\t ;|:]",engine='python')


column number 0 <class 'int'> 30
0        0
1        1
2        2
3        3
4        4
      ... 
117    117
118    118
119    119
120    120
121    121
Name: Unnamed: 0, Length: 122, dtype: int64
Can only use .str accessor with string values!
column number 1 <class 'int'> 30
0        0
1        1
2        2
3        3
4        4
      ... 
117    117
118    118
119    119
120    120
121    121
Name: Unnamed, Length: 122, dtype: int64
Can only use .str accessor with string values!
column number 2 <class 'int'> 30
0        0
1        1
2        2
3        3
4        4
      ... 
117    117
118    118
119    119
120    120
121    121
Name: Unnamed: 2, Length: 122, dtype: int64
Can only use .str accessor with string values!
column number 3 <class 'int'> 30
0              Plate
1                  1
2         Background
3              Plate
4                  1
           ...      
117         Nickname
118    Normalization
119            Assay
120          WARNING
121         Exported
Name

C:\Users\vandu\AppData\Local\Temp\ipykernel_16724\1409865649.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(folder+filename, on_bad_lines='skip', encoding = "utf-8",sep="[_,\t ;|:]")#,sep ="[_\t ;|:]",engine='python')
C:\Users\vandu\AppData\Local\Temp\ipykernel_16724\1409865649.py:37: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(folder+filename, on_bad_lines='skip', encoding = "utf-8",sep="[_,\t ;|:]")#,sep ="[_\t ;|:]",engine='python')


archive
completed
